# 实践实验室：线性回归

欢迎来到您的第一个实践实验室！在本实验中，您将使用一个变量实施线性回归来预测特许经营餐厅的利润。

# 大纲
- [1 包]()
- [2 单变量线性回归]()
   - [2.1 问题陈述]()
   - [2.2 数据集]()
   - [2.3 线性回归复习]()
   - [2.4 计算成本]()
     - [练习1]()
   - [2.5 梯度下降]()
     - [练习2]()
   - [2.6 使用批量梯度下降学习参数]()

## 1 套餐

首先，让我们运行下面的单元来导入您在此作业期间需要的所有包。

- [numpy](https://numpy.org/)是在Python中处理矩阵的基本包。
- [matplotlib](https://matplotlib.org)是一个著名的Python绘图库。
- `utils.py`包含此任务的辅助函数。您不需要修改此文件中的代码。

In [21]:
import matplotlib.pyplot as plt
from utils import *
import copy
import math
%matplotlib inline

## 2 问题陈述

假设您是一家特许经营餐厅的首席执行官，正在考虑在不同的城市开设新店。

- 您想将业务扩展到可以为您的餐厅带来更高利润的城市。
- 该连锁店已经在各个城市设有餐厅，并且您拥有这些城市的利润和人口数据。
- 您还拥有有关新餐厅候选城市的数据。
    - 对于这些城市，您有城市人口。
    
您能否使用这些数据来帮助您确定哪些城市可能为您的企业带来更高的利润？

## 3 数据集

您将首先加载此任务的数据集。

- 下面显示的`load_data()`函数将数据加载到变量`x_train`和`y_train`中
   - `x_train`是一个城市的人口
   - `y_train`是该城市一家餐馆的利润。 利润为负值表示亏损。
   - `x_train`和`y_train`都是NumPy数组。

In [22]:
x_train, y_train = load_data()

#### 查看变量

在开始任何任务之前，更熟悉您的数据集很有用。

- 一个好的起点是打印出每个变量并查看它包含的内容。

下面的代码打印变量`x_train`和变量的类型。

In [23]:
print("Type of x_train:", type(x_train))
print("First five elements of x_train are:\n", x_train[:5])

`x_train`是一个NumPy数组，其中包含全部大于零的十进制值。

- 这些值代表城市人口乘以10,000
- 例如，6.1101表示该城市的人口为61,101
  
现在，让我们打印`y_train`

In [24]:
print("Type of y_train:", type(y_train))
print("First five elements of y_train are:\n", y_train[:5])

类似地，`y_train`是一个NumPy数组，具有十进制值，有些是负值，有些是正值。

- 这些代表您的餐厅在每个城市的平均每月利润，单位为10,000美元。
   - 例如，17.592 代表该城市的平均月利润为175,920美元。
   - -2.6807代表该城市的平均每月损失26,807美元。

#### 检查变量的维度

熟悉数据的另一个有用方法是查看其维度。

请打印`x_train`和`y_train`的形状，并查看数据集中有多少个训练示例。

In [25]:
print('The shape of x_train is:', x_train.shape)
print('The shape of y_train is: ', y_train.shape)
print('Number of training examples (m):', len(x_train))

城市人口数组有97个数据点，月平均利润也有97个数据点。这些是NumPy一维数组。

#### 可视化您的数据

通过可视化数据来理解数据通常很有用。

- 对于此数据集，您可以使用散点图来可视化数据，因为它只有两个要绘制的属性（利润和人口）。
- 现实生活中你会遇到的许多其他问题都有两个以上的属性（例如人口、家庭平均收入、月利润、月销售额）。当你有两个以上的属性时，你仍然可以使用散点图来查看 每对属性之间的关系。

In [26]:
plt.scatter(x_train, y_train, marker='x', c='r')
plt.title("Profits vs. Population per city")
plt.ylabel('Profit in $10,000')
plt.xlabel('Population of City in 10,000s')
plt.show()

您的目标是构建一个线性回归模型来拟合这些数据。

- 通过此模型，您可以输入一个新城市的人口，并让该模型估计您的餐厅在该城市的每月潜在利润。

## 4 线性回归复习

在此练习实验室中，您将把线性回归参数$(w,b)$拟合到您的数据集。

- 线性回归的模型函数，这是一个从`x`（城市人口）映射到`y`（您的餐厅在该城市的月利润）的函数，表示为
  $$f_{w,b}(x) = wx + b$$
- 要训练线性回归模型，您需要找到适合您的数据集的最佳$(w,b)$参数。
    - 要比较$(w,b)$的一个选择比另一个选择更好或更差，您可以使用成本函数$J(w,b)$对其进行评估
        - $J$是$(w,b)$的函数。也就是说，成本$J(w,b)$的值取决于$(w,b)$的值。
    - 选择最适合您的数据的$(w,b)$是成本$J(w,b)$最小的那个。
- 要找到获得最小可能成本$J(w,b)$的值$(w,b)$，您可以使用称为**梯度下降**的方法。
    - 随着梯度下降的每一步，您的参数$(w,b)$更接近实现最低成本$J(w,b)$的最佳值。
- 经过训练的线性回归模型可以采用输入特征$x$（城市人口）并输出预测$f_{w,b}(x)$（该城市餐厅的预测月利润）。

## 5 计算成本

梯度下降涉及重复步骤来调整参数$(w,b)$的值，以逐渐获得越来越小的成本$J(w,b)$。

- 在梯度下降的每一步，通过计算$(w,b)$更新时的成本$J(w,b)$来监控进度将很有帮助。
- 在本节中，您将实现一个函数来计算$J(w,b)$，以便您可以检查梯度下降实现的进度。

#### 成本函数

您可能还记得讲座中的内容，对于一个变量，线性回归的成本函数$J(w,b)$定义为

$$J(w,b)=\frac{1}{2m}\sum\limits_{i=0}^{m-1}(f_{w,b}(x^{(i)})-y^{(i)})^2$$

- 您可以将$f_{w,b}(x^{(i)})$视为模型对餐厅利润的预测，而不是$y^{(i)}$，后者是记录在数据中的餐厅实际利润。
- $m$是数据集中训练样本的数量

#### 模型预测

- 对于一个变量的线性回归，模型$f_{w,b}$对于示例$x^{(i)}$的预测表示为：
  $$f_{w,b}(x^{(i)})=wx^{(i)}+b$$

这是直线方程，截距为$b$，斜率为$w$

#### 执行

请完成下面的`compute_cost()`函数来计算成本$J(w,b)$。

### 练习1

完成下面的`compute_cost`：

* 迭代训练示例，并针对每个示例计算：
    * 该示例的模型预测$f_{wb}(x^{(i)})=wx^{(i)}+b$
    * 该示例的成本$cost^{(i)}=(f_{wb}-y^{(i)})^2$
* 返回所有示例的总成本
  $J(\mathbf{w},b)=\frac{1}{2m}\sum\limits_{i=0}^{m-1}cost^{(i)}$
    * 这里，$m$是训练样本的数量，$\sum$是求和运算符

如果您遇到困难，可以查看下面单元格后面提供的提示来帮助您实施。

In [27]:
def compute_cost(x, y, w, b):
    m = x.shape[0]
    total_cost = 0
    for i in range(m):
        total_cost += (w * x[i] + b - y[i]) ** 2
    total_cost /= 2 * m
    return total_cost

您可以通过运行以下测试代码来检查您的实现是否正确：

In [28]:
initial_w = 2
initial_b = 1
cost = compute_cost(x_train, y_train, initial_w, initial_b)
print(type(cost))
print(f'Cost at initial w (zeros): {cost:.3f}')
from public_tests import *

compute_cost_test(compute_cost)

**预期输出**：

<table>
  <tr>
    <td> <b>Cost at initial w (zeros):<b> 75.203 </td> 
  </tr>
</table>

## 6 梯度下降

在本节中，您将为线性回归的参数$w、b$实现梯度。

正如讲座视频中所述，梯度下降算法为：

重复直到收敛：

$$
w=w-\alpha\frac{\partial J(w,b)}{\partial w}\tag{1}
$$

$$
b=b-\alpha\frac{\partial J(w,b)}{\partial b}
$$

其中，参数$w、b$同时更新，并且其中

$$
\begin{align}
\frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{2}\\
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{3}\\
\end{align}
$$

* m是数据集中训练样本的数量
* $f_{w,b}(x^{(i)})$是模型的预测，而$y^{(i)}$是目标值

您将实现一个名为`compute_gradient`的函数，该函数计算$\frac{\partial J(w)}{\partial w}$、$\frac{\partial J(w)}{\partial b}$    

### 练习2

请完成`compute_gradient`函数：

* 迭代训练示例，并针对每个示例计算：
    * 该示例的模型预测$f_{wb}(x^{(i)})=wx^{(i)}+b$
    * 该示例中参数$w$，$b$的梯度
      $$
      \frac{\partial J(w,b)}{\partial b}^{(i)}=(f_{w,b}(x^{(i)})-y^{(i)})
      $$
      $$
      \frac{\partial J(w,b)}{\partial w}^{(i)}=(f_{w,b}(x^{(i)})-y^{(i)})x^{(i)}
      $$
* 返回所有示例的总梯度更新    
  $$
  \frac{\partial J(w,b)}{\partial b}=\frac{1}{m}\sum\limits_{i = 0}^{m-1}\frac{\partial J(w,b)}{\partial b}^{(i)}
  $$
  $$
  \frac{\partial J(w,b)}{\partial w}=\frac{1}{m}\sum\limits_{i = 0}^{m-1}\frac{\partial J(w,b)}{\partial w}^{(i)}
  $$
  * 这里，$m$是训练样本的数量，$\sum$是求和运算符

如果您遇到困难，可以查看下面单元格后面提供的提示来帮助您实施。

In [29]:
def compute_gradient(x, y, w, b):
    m = x.shape[0]
    dj_dw = 0
    dj_db = 0
    for i in range(m):
        dj_db += w * x[i] + b - y[i]
        dj_dw += (w * x[i] + b - y[i]) * x[i]
    dj_db /= m
    dj_dw /= m
    return dj_dw, dj_db

运行下面的单元格来检查`compute_gradient`函数的实现，其中参数`w`、`b`有两种不同的初始化。

In [30]:
initial_w = 0
initial_b = 0
tmp_dj_dw, tmp_dj_db = compute_gradient(x_train, y_train, initial_w, initial_b)
print('Gradient at initial w, b (zeros):', tmp_dj_dw, tmp_dj_db)
compute_gradient_test(compute_gradient)

现在让我们在我们的数据集上运行上面实现的梯度下降算法。

**预期输出**：

<table>
  <tr>
    <td> <b>Gradient at initial , b (zeros)<b></td>
    <td> -65.32884975 -5.83913505154639</td> 
  </tr>
</table>

In [31]:
test_w = 0.2
test_b = 0.2
tmp_dj_dw, tmp_dj_db = compute_gradient(x_train, y_train, test_w, test_b)
print('Gradient at test w, b:', tmp_dj_dw, tmp_dj_db)

**预期输出**：
<table>
  <tr>
    <td> <b>Gradient at test w<b></td>
    <td> -47.41610118 -4.007175051546391</td> 
  </tr>
</table>

### 2.6 使用批量梯度下降学习参数

现在，您将通过使用批量梯度下降找到线性回归模型的最佳参数。召回批次是指在一次迭代中运行所有示例。

- 您不需要为此部分实现任何内容。只需运行下面的单元格即可。
- 验证梯度下降是否正常工作的一个好方法是查看$J(w,b)$的值并检查它是否随着每一步而减少。
- 假设您已经实现了梯度并正确计算了成本，并且您有适当的学习率$\alpha$值，$J(w,b)$应该永远不会增加，并且应该在算法结束时收敛到稳定值。

In [32]:
def gradient_descent(x, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    J_history = []
    w_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    for i in range(num_iters):
        dj_dw, dj_db = gradient_function(x, y, w, b)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        if i < 100000:
            cost = cost_function(x, y, w, b)
            J_history.append(cost)
        if i % math.ceil(num_iters / 10) == 0:
            w_history.append(w)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
    return w, b, J_history, w_history

现在让我们运行上面的梯度下降算法来学习数据集的参数。

In [33]:
initial_w = 0.
initial_b = 0.
iterations = 1500
alpha = 0.01
w, b, _, _ = gradient_descent(x_train, y_train, initial_w, initial_b, compute_cost, compute_gradient, alpha, iterations)
print("w,b found by gradient descent:", w, b)

**预期输出**：
<table>
  <tr>
    <td> <b> w, b found by gradient descent<b></td>
    <td> 1.16636235 -3.63029143940436</td> 
  </tr>
</table>

我们现在将使用梯度下降的最终参数来绘制线性拟合。

回想一下，我们可以获得单个示例$f(x^{(i)})=wx^{(i)}+b$的预测。

为了计算整个数据集的预测，我们可以循环遍历所有训练示例并计算每个示例的预测。这如下面的代码块所示。

In [34]:
m = x_train.shape[0]
predicted = np.zeros(m)
for i in range(m):
    predicted[i] = w * x_train[i] + b

我们现在将绘制预测值以查看线性拟合。

In [35]:
plt.plot(x_train, predicted, c="b")
plt.scatter(x_train, y_train, marker='x', c='r')
plt.title("Profits vs. Population per city")
plt.ylabel('Profit in $10,000')
plt.xlabel('Population of City in 10,000s')

您的最终值$w,b$也可用于预测利润。让我们预测一下35,000人和70,000人的地区的利润是多少。

- 该模型将一个城市的数万人口作为输入。
- 因此，35,000人可以转化为模型的输入为`np.array([3.5])`
- 同样，70,000人可以转换为模型的输入，如`np.array([7.])`

In [36]:
predict1 = 3.5 * w + b
print('For population = 35,000, we predict a profit of $%.2f' % (predict1 * 10000))
predict2 = 7.0 * w + b
print('For population = 70,000, we predict a profit of $%.2f' % (predict2 * 10000))

**预期输出**:

<table>
    <tr>
        <td>
            <b>
                For population = 35,000, we predict a profit of
            </b>
        </td>
        <td>
            $4519.77
        </td> 
    </tr>
    <tr>
        <td> 
            <b> 
                For population = 70,000, we predict a profit of
            </b>
        </td>
        <td>
            $45342.45
        </td> 
     </tr>
</table>